# Text Classification and Recurrent Neural Networks



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip install datasets

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2


    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2


  Attempting uninstall: requests


    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0


  Attempting uninstall: fsspec


    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0


Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset

ds = load_dataset("Paul/hatecheck")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generating test split:   0%|   | 0/3728 [00:00<?, ? examples/s]

Generating test split: 100%|█| 3728/3728 [00:00<00:00, 132866.2

In [4]:
df = ds['test'].to_pandas()

In [5]:
df['test_case'].head()

0              I hate women. 
1       I hate trans people. 
2         I hate gay people. 
3       I hate black people. 
4    I hate disabled people. 
Name: test_case, dtype: object

In [6]:
df['label_gold'].value_counts()

label_gold
hateful        2563
non-hateful    1165
Name: count, dtype: int64

### Encoding the Text



In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cvect = CountVectorizer()

In [9]:
dtm = cvect.fit_transform(df['test_case'])

In [10]:
pd.DataFrame(dtm.toarray(), columns=cvect.get_feature_names_out()).head()

,2020,4ssholes,abhor,about,absolute,absolutefilth,absolutely,academics,accepted,accountants,...,writing,wrong,yeah,years,you,your,yours,yourself,yourselves,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = dtm
y = df['label_gold']

### Problem

Split the data and build a random forest classifier on the training data.  Compare the train and test scores.  

- What words were most important in making the classifications?
- What elements of the CountVectorizer might you change or grid search to attempt to improve your model?

# Text Classification with Neural Networks

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### Classifying Spam

In [ ]:
#read in data
spam = pd.read_csv('sms_spam.csv')

In [ ]:
#take a peek
spam.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...


In [ ]:
#create a tokenizer
tokenizer = Tokenizer(num_words = 500)

In [ ]:
#fit the tokenizer
tokenizer.fit_on_texts(spam['text'].values)

In [ ]:
#look at tokenizer
tokenizer.num_words

500

In [ ]:
#create document term matrix (binarized)
dtm = tokenizer.texts_to_matrix(spam['text'].values)

In [ ]:
#take a peek
dtm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
spam['text'][2]

'Am also doing in cbe only. But have to pay.'

In [ ]:
class TextDataset(Dataset):
  def __init__(self, X, y):
    super().__init__()
    self.x = torch.tensor(X, dtype = torch.float)
    self.y = torch.tensor(y, dtype = torch.float)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
X = dtm
y = np.where(spam['type'] == 'ham', 0, 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
X_train

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
#create data class
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

In [ ]:
#dataset and loader
trainloader = DataLoader(train_dataset, batch_size = 32)
#dataset and loader
testloader = DataLoader(test_dataset, batch_size = 32)

In [ ]:
#loss and optimizer
class TextModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lin1 = nn.Linear(in_features = 500, out_features = 100)
    self.lin2 = nn.Linear(100, 100)
    self.lin3 = nn.Linear(100, 1)
    self.sigmoid = nn.Sigmoid()
    self.act = nn.ReLU()

  def forward(self, x):
    x = self.act(self.lin1(x))
    x = self.act(self.lin2(x))
    return self.sigmoid(self.lin3(x))




In [ ]:
#training function
model = TextModel()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [ ]:
#evaluate
for epoch in range(100):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

Epoch 0 Loss: 17.463785945263226
Epoch 10 Loss: 0.3230317887267802
Epoch 20 Loss: 0.1858880319979619
Epoch 30 Loss: 0.18419960872658825
Epoch 40 Loss: 0.19837459240657174
Epoch 50 Loss: 0.18659224085559767
Epoch 60 Loss: 45.2216811845803
Epoch 70 Loss: 6.432060421328064
Epoch 80 Loss: 6.417951953306446
Epoch 90 Loss: 6.412450148929762


In [ ]:
Xt = torch.tensor(X_test, dtype = torch.float)

In [ ]:

output = model(Xt) #model predictions

In [ ]:
output

tensor([[2.1603e-23],
        [4.3468e-08],
        [2.5849e-02],
        ...,
        [0.0000e+00],
        [0.0000e+00],
        [1.3465e-31]], grad_fn=<SigmoidBackward0>)

In [ ]:
#Converting probabilities to prediction
preds = np.where(np.array(output.detach()) >= .5, 1, 0)

In [ ]:
preds.shape

(1112, 1)

In [ ]:
y = np.where(spam['type'] == 'ham', 0, 1)

In [ ]:
sum(preds[:, 0] == y_test)/len(y_test)

0.9784172661870504

In [ ]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([972, 140]))

In [ ]:
951/(951 + 161)

0.8552158273381295

### Basic RNN

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/440px-Recurrent_neural_network_unfold.svg.png)

In [ ]:
#create sequences
sequences = tokenizer.texts_to_sequences(spam['text'].values)

In [ ]:
#look at first sequence
sequences[0]

[122, 3, 22, 313, 4, 53, 110, 37, 8]

In [ ]:
#compare to text
spam['text'].values[1]

'K..give back my thanks.'

In [ ]:
#pad and make all same length
sequences = pad_sequences(sequences, maxlen=100)

In [ ]:
#examine results
sequences[1].shape

(100,)

In [ ]:
sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,  92, 134,  86,  11, 170], dtype=int32)

In [ ]:
#example rnn
rnn = nn.RNN(input_size = 100,
             hidden_size = 30,
             num_layers = 1,
             batch_first = True)

In [ ]:
#pass data through
sample_sequence = torch.tensor(sequences[1],
                               dtype = torch.float,
                               ).reshape(1, -1)
sample_sequence.shape

torch.Size([1, 100])

In [ ]:
#output
output, hidden = rnn(sample_sequence)

In [ ]:
#hidden
hidden

tensor([[ 1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -0.9969,
          1.0000,  1.0000,  1.0000,  0.9927, -0.9988,  1.0000,  1.0000, -1.0000,
         -1.0000,  0.9980, -0.9968,  1.0000,  0.8639, -1.0000, -1.0000,  0.9869,
          1.0000,  0.7625,  1.0000,  0.6461, -1.0000, -0.9947]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
#linear layer
output.shape

torch.Size([1, 30])

In [ ]:
#pass through linear
lin1 = nn.Linear(in_features = 30, out_features = 1)

In [ ]:
lin1(output)

tensor([[0.1402]], grad_fn=<AddmmBackward0>)

In [ ]:
for x, y in trainloader:
  print(x.shape)
  break

torch.Size([32, 500])


In [ ]:
class TextDataset(Dataset):
  def __init__(self, X, y):
    super().__init__()
    self.x = torch.tensor(X, dtype = torch.float)
    self.y = torch.tensor(y, dtype = torch.float)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
#class
class BasicRNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.RNN(input_size = 100,
                    hidden_size = 100,
                    num_layers = 3,
                    batch_first = True)
    self.lin1 = nn.Linear(in_features = 100, out_features=1000)
    self.lin2 = nn.Linear(1000, 100)
    self.lin3 = nn.Linear(100, 1)
    self.act = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.act(self.lin1(x))
    x = self.act(self.lin2(x))
    x = self.sigmoid(self.lin3(x))
    return x


In [ ]:
#data
X = sequences
y = np.where(spam['type'] == 'spam', 1, 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
traindata = TextDataset(X_train, y_train)
trainloader = DataLoader(traindata, batch_size = 32)

In [ ]:
#optimizer and loss
model = BasicRNN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [ ]:
#train
for epoch in range(100):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

Epoch 0 Loss: 54.333703458309174
Epoch 10 Loss: 49.14352545142174
Epoch 20 Loss: 50.294177405536175
Epoch 30 Loss: 54.339055240154266
Epoch 40 Loss: 54.340724781155586
Epoch 50 Loss: 54.34115116298199
Epoch 60 Loss: 54.34125591814518
Epoch 70 Loss: 54.341282427310944
Epoch 80 Loss: 54.34128923714161
Epoch 90 Loss: 54.34129001200199


In [ ]:
Xt = torch.tensor(X_test, dtype = torch.float)

In [ ]:
output = model(Xt)

In [ ]:
preds = np.where(np.array(output.detach()) >= .5, 1, 0)

In [ ]:
#preds = output.argmax(axis = 1)

In [ ]:
y_test

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
# y = np.where(spam['type'] == 'ham', 0, 1)

In [ ]:
# y.shape

In [ ]:
sum(preds.reshape(1112,) == y_test)/len(y_test)

0.8588129496402878

### Pretrained Models and HuggingFace

- [Huggingface](https://huggingface.co/)
- [Chronos Paper](https://arxiv.org/abs/2403.07815)

In [ ]:
pip install git+https://github.com/amazon-science/chronos-forecasting.git

In [ ]:
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-large",
  device_map="cuda",
  torch_dtype=torch.bfloat16,
)

df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")


In [ ]:
# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
context = torch.tensor(df["#Passengers"])
prediction_length = 12
forecast = pipeline.predict(context, prediction_length)  # shape [num_series, num_samples, prediction_length]

In [ ]:
# visualize the forecast
forecast_index = range(len(df), len(df) + prediction_length)
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

plt.figure(figsize=(8, 4))
plt.plot(df["#Passengers"], color="royalblue", label="historical data")
plt.plot(forecast_index, median, color="tomato", label="median forecast")
plt.fill_between(forecast_index, low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid();

### Problem

Explore the pretrained models available and try to find one that is either of relevance to your final paper or just of general interest.  Load and use the model in an example -- even just the docs!

#### LSTM

In [ ]:
# nn.LSTM()
class BasicLSTM(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.LSTM(input_size = 100,
                    hidden_size = 100,
                    num_layers = 1,
                    batch_first = True)

    self.lin1 = nn.Linear(in_features = 100, out_features=100)
    self.lin2 = nn.Linear(in_features = 100, out_features = 1)
    self.act = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.act(self.lin1(x))
    x = self.lin2(x)
    return self.sigmoid(x)

In [ ]:
model = BasicLSTM()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [ ]:
#train
for epoch in range(10):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

Epoch 0 Loss: 49.52611651271582


In [ ]:
Xt = torch.tensor(X_test, dtype = torch.float)
output = model(Xt)
preds = np.where(np.array(output.detach()) >= .5, 1, 0)
sum(preds[:, 0] == y_test)/len(y_test)

In [ ]:
#pad and make all same length
sequences = pad_sequences(sequences, maxlen=30)

In [ ]:
sequences[0]

In [ ]:
X = sequences
y = np.where(spam['type'] == 'spam', 1, 0)
data = TextDataset(X, y)
loader = DataLoader(data, batch_size = 32)

In [ ]:
class RNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.LSTM(input_size = 30,
                    hidden_size = 30,
                    num_layers = 2,
                    batch_first = True)

    self.lin1 = nn.Linear(in_features = 30, out_features=100)
    self.lin2 = nn.Linear(in_features = 100, out_features = 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.lin1(x)
    x = self.lin2(x)
    return self.sigmoid(x)

In [ ]:
model = RNN2()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [ ]:
#train
for epoch in range(100):
  losses = 0
  for x,y in loader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

In [ ]:
Xt = torch.tensor(sequences, dtype = torch.float)
output = model(Xt)
preds = np.where(np.array(output.detach()) >= .5, 1, 0)
y = np.where(spam['type'] == 'ham', 0, 1)
sum(preds[:, 0] == y)/len(y)

In [ ]:
y

In [ ]:
output

In [ ]:
spam['type']

In [ ]:
preds.sum()

In [ ]:
(y == 1).sum()